In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rcParams
import os
import sys
import json

In [ ]:
path_to_data = os.path.join("..", "raw_data", "NGSIM", "Lankershim-Boulevard-LosAngeles-CA", 
                            "signal-timing", "real-time-split-monitor-reports", "Original Text files")
path_to_output = os.path.join("..", "data", "NGSIM")
signal_input_file_name = "RSM0{0}-061605.txt"
dic_file_name_mapping = {
    "Intersection_1_1": signal_input_file_name.format(87),
    "Intersection_1_2": signal_input_file_name.format(88),
    "Intersection_1_3": signal_input_file_name.format(89),
    "Intersection_1_4": signal_input_file_name.format(90),
}
signal_output_file_name = "flow_la.json"

In [ ]:
start_time = pd.Timestamp("08:28:00")
end_time = pd.Timestamp("09:03:00")

In [ ]:
# original

dic_signal_setting_original = {
    # 87
    "Intersection_1_1": {
        # phase_no: ([(e1, l1), (e2, l2), ...], yellow_time, red_rime),
        2: ([("N", "s"),], 3.5, 0.5),
        4: ([("E", "l"), ("E", "r"),], 3.5, 0.5),
        6: ([("S", "s"),], 3.5, 0.5),
    },
    
    # 88
    "Intersection_1_2": {
        1: ([("N", "l"), ("E", "r"),], 3, 1),
        2: ([("S", "s"),], 4, 1.5),
        3: ([("E", "s"), ("E", "l"), ("S", "r"),], 3.5, 2.5),
        4: ([("W", "s"),], 3.5, 2.5),
        5: ([("S", "l"),], 3, 1),
        6: ([("N", "s"),], 4, 1.5),
    },
    
    # 89
    "Intersection_1_3": {
        2: ([("N", "s"),], 3.5, 1),
        3: ([("E", "s"), ("E", "l"),], 3.5, 1),
        4: ([("W", "s"),], 3.5, 1.5),
        5: ([("N", "l"),], 3, 0.5),
        6: ([("S", "s"),], 3.5, 1),
    },
    
    # 90
    "Intersection_1_4": {
        2: ([("N", "s"),], 3.5, 1),
        3: ([("E", "s"), ("E", "l"),], 3.5, 1),
        4: ([("W", "s"), ("W", "l"),], 3.5, 2),
        5: ([("N", "l"), ("E", "r"),], 3, 0.5),
        6: ([("S", "s"),], 3.5, 1),
    },
}

dic_signal_sync = {
    "Intersection_1_1": [
        ([2,], [6,],),
    ],
    "Intersection_1_2": [
        ([1, 2,], [5, 6,],),
    ],
    "Intersection_1_3": [
        ([2,], [5, 6,],),
    ],
    "Intersection_1_4": [
        ([2,], [5, 6,],),
    ],
}

# modified

dic_signal_setting = {
    # 87
    "Intersection_1_1": {
        # phase_no: ([(e1, l1), (e2, l2), ...], yellow_time, red_rime),
        2: ([("N", "s"),], 4),
        4: ([("E", "l"), ("E", "r"),], 4),
        6: ([("S", "s"),], 4),
    },
    
    # 88
    "Intersection_1_2": {
        1: ([("N", "l"), ("E", "r"),], 4),
        2: ([("S", "s"),], 6),
        3: ([("E", "s"), ("E", "l"), ("S", "r"),], 6),
        4: ([("W", "s"),], 6),
        5: ([("S", "l"),], 4),
        6: ([("N", "s"),], 6),
        8: ([], 4),
    },
    
    # 89
    "Intersection_1_3": {
        2: ([("N", "s"),], 4),
        3: ([("E", "s"), ("E", "l"),], 4),
        4: ([("W", "s"),], 5),
        5: ([("N", "l"),], 4),
        6: ([("S", "s"),], 4),
        8: ([], 5),
    },
    
    # 90
    "Intersection_1_4": {
        2: ([("N", "s"),], 5),
        3: ([("E", "s"), ("E", "l"),], 4),
        4: ([("W", "s"), ("W", "l"),], 5),
        5: ([("N", "l"), ("E", "r"),], 4),
        6: ([("S", "s"),], 4),
    },
}


In [ ]:
# width
# [8, 2, 2, 3, 3,    3, 3, 3, 3, 3, 3, 3, 3]
dic_df = {}
for inter_name, file_name in dic_file_name_mapping.items():
    dic_df[inter_name] = pd.read_fwf(os.path.join(path_to_data, file_name), skiprows=8, skipfooter=3, 
                     widths=[8, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], 
                     header=1)
    dic_df[inter_name].drop(0, inplace=True)
    dic_df[inter_name].drop("ST", axis=1, inplace=True)
    dic_df[inter_name]["TIME"] = dic_df[inter_name]["TIME"].apply(pd.to_datetime)

In [ ]:
def get_phase_time(r, dic_red, cyc, inter_name):
    dic = {}
    for i in range(1, 9):
        pt = r[str(i)]
        if pt != 0:
            if pt == cyc: # only this phase is set in this cycle
                dic[i] = [pt, 0]
            else:
                dic[i] = [pt, dic_red[i][1]]
        
    # check sync
    for pair in dic_signal_sync[inter_name]:
        group1, group2 = pair
        pts1 = sum([sum(dic[i]) if i in dic else 0 for i in group1])
        pts2 = sum([sum(dic[i]) if i in dic else 0 for i in group2])
        if pts1 > pts2:
            dic[group1[-1]][1] -= (pts1-pts2)
            assert dic[group1[-1]][1] > 0
        elif pts1 < pts2:
            dic[group2[-1]][1] -= (pts2-pts1)
            assert dic[group2[-1]][1] > 0

    cyc1 = np.sum([sum(pt) for i, pt in dic.items() if i <=4])
    cyc2 = np.sum([sum(pt) for i, pt in dic.items() if i >=5])

    if cyc1 != cyc:
        for i in [4, 3, 2, 1]:
            if i in dic:
                dic[i][1] += (cyc-cyc1)
                break
        assert dic[i][1] > 0

    if cyc2 != cyc:
        for i in [8, 7, 6, 5]:
            if i in dic:
                dic[i][1] += (cyc-cyc2)
                break
        assert dic[i][1] > 0
        
    return dic

dic_signal_timing = {}
for inter_name, dic_red_time in dic_signal_setting.items():
    print(inter_name)
    dic_signal_timing[inter_name] = {}
    df = dic_df[inter_name]
    df_sub = df[(df["TIME"] >= start_time - pd.Timedelta("180 seconds"))
        & (df["TIME"] <= end_time + pd.Timedelta("180 seconds"))]
    # parse every cycle to every second
    for i in range(len(df_sub)):
        r = df_sub.iloc[i]
        t_start = r["TIME"]
        cyc = r["CYC"]
        dic_phase_time = get_phase_time(r, dic_red_time, cyc, inter_name)
        dic_signal_timing[inter_name][t_start] = {"cycle": cyc, "timing": dic_phase_time}

# check
for inter_name, signal_timing in dic_signal_timing.items():
    df = dic_df[inter_name]
    for ts, sig in signal_timing.items():
        cycle = sig["cycle"]
        timing = sig["timing"]
        for i, pt in timing.items():
            if df[df["TIME"] == ts][str(i)].values[0] != pt[0]:
                print("error")
                break
print("correct")

In [ ]:
df[df["TIME"] == ts][str(i)].values[0]

In [ ]:
dic_signal_by_seconds = {}
for inter_name, signal_timing in dic_signal_timing.items():
    print(inter_name)
    dic_signal_by_seconds[inter_name] = {}
    t = start_time - pd.Timedelta("180 seconds")
    while t < end_time + pd.Timedelta("360 seconds"):
        dic_signal_by_seconds[inter_name][t] = []
        t += pd.Timedelta("1 seconds")
    for ts, sig in signal_timing.items():
        cycle = sig["cycle"]
        timing = sig["timing"]
        # 1, 2, 3, 4
        cnt_1 = 0
        for p_id in range(1, 5):
            if p_id in timing:
                p_time = timing[p_id]
                for j in range(int(p_time[0])):
                    cnt_1 += 1
                    dic_signal_by_seconds[inter_name][ts + pd.Timedelta("{0} seconds".format(cnt_1))] += dic_signal_setting[inter_name][p_id][0]
                cnt_1 += p_time[1]
        cnt_2 = 0
        for p_id in range(5, 9):
            if p_id in timing:
                p_time = timing[p_id]
                for j in range(int(p_time[0])):
                    cnt_2 += 1
                    dic_signal_by_seconds[inter_name][ts + pd.Timedelta("{0} seconds".format(cnt_2))] += dic_signal_setting[inter_name][p_id][0]
                cnt_2 += p_time[1]

        assert cnt_1 == cycle
        assert cnt_2 == cycle, print(timing, p_id)

In [ ]:
dic_signal_by_seconds["Intersection_1_1"]

In [ ]:
def convert_phase_tuple_to_str(phase):
    list_s = []
    for s in phase:
        list_s += list(s)
    signal_str = ''.join(list_s)
    return signal_str

dic_unique_phase = {}
dic_unique_phase_reverse = {}
for inter_name, signal_by_seconds in dic_signal_by_seconds.items():
    dic_unique_phase[inter_name] = {0: "all red"}
    dic_unique_phase_reverse[inter_name] = {"all red": 0}
    list_signal_str = []
    phase_no = 1
    for ts, phase in signal_by_seconds.items():
        signal_str = convert_phase_tuple_to_str(phase)
        if signal_str != '' and (signal_str not in list_signal_str):
            dic_unique_phase[inter_name][phase_no] = phase
            dic_unique_phase_reverse[inter_name][signal_str] = phase_no
            list_signal_str.append(signal_str)
            phase_no += 1 
            

In [ ]:
json.dump(dic_unique_phase, open(os.path.join(path_to_output, "phase_setting.json"), "w"), indent=2)

In [ ]:
t = start_time
list_time = []
dic_aggregate = {
    "Intersection_1_1": [],
    "Intersection_1_2": [],
    "Intersection_1_3": [],
    "Intersection_1_4": [],
}
while t <= end_time:
    list_time.append(t)
    for inter_name, signal_by_seconds in dic_signal_by_seconds.items():
        signal_str = convert_phase_tuple_to_str(signal_by_seconds[t])
        dic_aggregate[inter_name].append(dic_unique_phase_reverse[inter_name][signal_str] if signal_str in dic_unique_phase_reverse[inter_name] else 0
            )
    t += pd.Timedelta("1 seconds")
df_signal_out = pd.concat([
    pd.Series(list_time, name="ts"), 
    pd.Series(dic_aggregate["Intersection_1_1"], name="Intersection_1_1"), 
    pd.Series(dic_aggregate["Intersection_1_2"], name="Intersection_1_2"),
    pd.Series(dic_aggregate["Intersection_1_3"], name="Intersection_1_3"),
    pd.Series(dic_aggregate["Intersection_1_4"], name="Intersection_1_4")
], axis=1)
    

In [ ]:
df_signal_out["time"] = pd.Series(list(range(0, len(df_signal_out))), index=df_signal_out.index)

In [ ]:
df_signal_out.drop("ts", axis=1, inplace=True)

In [ ]:
df_signal_out.to_pickle(os.path.join(path_to_output, "signal_la.json"))